In [ ]:
import pandas as pd
import numpy as np

# finding betas with final dataset

In [ ]:
migration_stats = pd.read_csv('../data/all_emigrants_high_interpolated.csv').drop(['Unnamed: 0','country_to'], axis=1)
years_data = migration_stats.iloc[:, 1:32].columns
migration_stats_by_origin = migration_stats.groupby('country_from')[years_data].sum()
migration_stats_by_origin = migration_stats_by_origin.diff(periods=1,axis=1)
migration_stats_by_origin = migration_stats_by_origin.fillna(0)
migration_stats_by_origin = migration_stats_by_origin.reset_index()
migration_stats_by_origin = migration_stats_by_origin.melt('country_from', var_name = 'year', value_name = 'num_of_emmigrants')
migration_stats_by_origin = migration_stats_by_origin.set_index(['country_from', 'year'])
migration_stats_by_origin

In [ ]:
migration_stats = pd.read_csv('../data/all_emigrants_high_interpolated.csv').drop('Unnamed: 0', axis=1)
migration_stats.head()

In [ ]:
years_data = migration_stats.iloc[:, 1:32].columns
migration_stats_by_destination = migration_stats.groupby('country_to')[years_data].sum()
migration_stats_by_destination

In [ ]:
migration_stats_by_destination['total_to_country'] = migration_stats_by_destination.sum(axis=1)
total_by_year = migration_stats_by_destination.sum(axis=0)
country_to = list(migration_stats_by_destination.reset_index()['country_to'])
country_to.append('total')
migration_stats_by_destination = migration_stats_by_destination.append(total_by_year, ignore_index=True)
migration_stats_by_destination['country_to'] = pd.Series(country_to)
# migration_stats_by_destination

In [ ]:
for col in migration_stats_by_destination.iloc[:, :-1].columns:
     migration_stats_by_destination[col][:-1] /= migration_stats_by_destination[col][len(migration_stats_by_destination)-1]

In [ ]:
migration_stats_by_destination_melted = migration_stats_by_destination.melt('country_to', var_name = 'year', value_name = 'prob')
migration_stats_by_destination_melted = migration_stats_by_destination_melted.set_index(['country_to', 'year'])

In [ ]:
country_data = pd.read_csv('../df_for_final_sim.csv').drop('Unnamed: 0', axis=1)
country_data = country_data.dropna()

country_data = country_data.rename({'country':'country'}, axis=1).set_index(['country','year'])
country_data

In [ ]:
migration_stats_by_destination_melted = migration_stats_by_destination_melted.reset_index()
migration_stats_by_destination_melted['merge_index'] = migration_stats_by_destination_melted['country_to'] +  migration_stats_by_destination_melted['year'].astype(str)

In [ ]:
Calibration_data_df = country_data.loc[migration_stats['country_from'].unique(),:]
Calibration_data_df

In [ ]:
country_data = country_data.reset_index()
country_data['merge_index'] = country_data['country'] +  country_data['year'].astype(str)

In [ ]:
Regression_data_df = pd.merge(migration_stats_by_destination_melted,country_data,how='inner',on='merge_index')

# Regression

In [ ]:
X = Regression_data_df.drop(['country_to','prob','year_y','merge_index'],axis=1)
X

In [ ]:
X.year_x = X.year_x.astype(int)
X.year_x = X.year_x-1980

In [ ]:
import statsmodels.formula.api as smf 

In [ ]:
Calibration_data_df = X.copy() 
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
Calibration_data_scaled_df = Calibration_data_df.copy()
Calibration_data_scaled_df[['co2', 'expRD', 'expEd', 'expHealth', 'gdp']] = mms.fit_transform(
    Calibration_data_scaled_df[['co2', 'expRD', 'expEd', 'expHealth', 'gdp']])

In [ ]:
Calibration_data_scaled_df ['total'] = Calibration_data_scaled_df[['co2', 'expRD', 'expEd', 'expHealth', 'gdp']].sum(axis=1)
Calibration_data_scaled_df

In [ ]:
us = Calibration_data_scaled_df.loc['Albania'] - Calibration_data_scaled_df.loc['United States']
canada = Calibration_data_scaled_df.loc['Albania'] - Calibration_data_scaled_df.loc['Canada']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
Regression_data_scaled_df = Regression_data_df.copy()
Regression_data_scaled_df[['co2', 'expRD', 'expEd', 'expHealth', 'gdp']] = mms.fit_transform(Regression_data_scaled_df[['co2', 'expRD', 'expEd', 'expHealth', 'gdp']])

In [ ]:
lin_mod = smf.ols ('prob~co2+expRD+expEd+expHealth+gdp', data = Regression_data_df).fit()

In [ ]:
lin_mod.params